In [2]:
pip install requests beautifulsoup4 pandas openpyxl pykrx lxml

^C
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# 크롤러

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from pykrx import stock
from urllib.parse import quote
from datetime import datetime

# pandas 출력 설정
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 200)

# 사용자 입력 받기
input_name = input("크롤링할 종목명을 입력하세요 (예: 카카오): ").strip()
start_date_input = input("시작일을 입력하세요 (예: 2023-07-01): ").strip()
end_date_input = input("종료일을 입력하세요 (예: 2025-07-25): ").strip()

# 날짜 형식 변환
start_date = datetime.strptime(start_date_input, "%Y-%m-%d")
end_date = datetime.strptime(end_date_input, "%Y-%m-%d")

# 종목명 → 종목코드 변환
ticker_list = stock.get_market_ticker_list(market="ALL")
ticker_name_map = {stock.get_market_ticker_name(code): code for code in ticker_list}

if input_name not in ticker_name_map:
    print(f"[오류] '{input_name}'에 해당하는 종목코드를 찾을 수 없습니다.")
else:
    item_code = ticker_name_map[input_name]
    base_url = "https://finance.naver.com/research/"
    headers = {"User-Agent": "Mozilla/5.0"}

    data = []

    for i in range(1, 7):  # 페이지 수 조정 가능
        url = f"https://finance.naver.com/research/company_list.naver?searchType=itemCode&itemName={quote(input_name)}&itemCode={item_code}&page={i}"
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        rows = soup.select("table.type_1 tr")[2:]

        for row in rows:
            cols = row.find_all("td")
            if len(cols) < 5:
                continue

            a_tag = cols[1].find("a", href=True)
            if not a_tag or not a_tag["href"].startswith("company_read.naver"):
                continue

            title = a_tag.text.strip()
            detail_url = base_url + a_tag["href"]
            broker = cols[2].text.strip()
            pdf_tag = cols[3].find("a", href=True)
            pdf_link = pdf_tag["href"] if pdf_tag and pdf_tag["href"].endswith(".pdf") else None
            date_str = cols[4].text.strip()

            # 작성일 필터링
            try:
                date = datetime.strptime(date_str, "%y.%m.%d")
            except:
                continue

            if not (start_date <= date <= end_date):
                continue

            # 상세 페이지 크롤링 (목표가, 투자의견)
            try:
                detail_res = requests.get(detail_url, headers=headers)
                detail_soup = BeautifulSoup(detail_res.text, "html.parser")

                target_price = detail_soup.select_one("em.money")
                investment_opinion = detail_soup.select_one("em.coment")

                target_price_text = target_price.text.strip() if target_price else None
                opinion_text = investment_opinion.text.strip() if investment_opinion else None
            except:
                target_price_text = None
                opinion_text = None

            data.append({
                "제목": title,
                "상세페이지링크": detail_url,
                "증권사": broker,
                "PDF 링크": pdf_link,
                "작성일": date_str,
                "목표가": target_price_text,
                "투자의견": opinion_text
            })

            time.sleep(0.3)

    # DataFrame 생성 (링크 있는 버전)
    df_link = pd.DataFrame(data)

    # Excel 저장 (링크 있는 버전)
    filename = f"{input_name}_{start_date_input}~{end_date_input}_리포트 수집.xlsx"
    df_link.to_excel(filename, index=False, engine='openpyxl')
    print(f"\n✅ 엑셀 파일로 저장 완료: {filename}")

    # DataFrame 생성 (링크 없는 버전)
    df_2 = pd.DataFrame(data)
    df_2 = df_2.drop(columns=["상세페이지링크", "PDF 링크"])


✅ 엑셀 파일로 저장 완료: 셀트리온_2023-07-01~2024-07-31_리포트 수집.xlsx


# 실제 주가 불러오기

In [3]:
from pykrx import stock
from datetime import timedelta

# 작성일 → datetime 변환
df_2["작성일_dt"] = pd.to_datetime(df_2["작성일"], format="%y.%m.%d")

# 1년 뒤 날짜 계산
df_2["1년뒤_예측일"] = df_2["작성일_dt"] + pd.DateOffset(years=1)

# 종목코드 다시 확인
item_code = ticker_name_map[input_name]

# 1년 뒤 종가 저장용 리스트
actual_prices = []

for idx, row in df_2.iterrows():
    predict_day = row["1년뒤_예측일"]
    from_date = predict_day.strftime("%Y%m%d")
    to_date = (predict_day + timedelta(days=5)).strftime("%Y%m%d")  # 영업일이 아닐 경우 대비
    
    try:
        df_price = stock.get_market_ohlcv_by_date(from_date, to_date, item_code)
        df_price = df_price.reset_index()
        
        # 가장 가까운 날짜의 종가 선택
        if not df_price.empty:
            price = df_price.iloc[0]["종가"]
        else:
            price = None
    except:
        price = None

    actual_prices.append(price)
    time.sleep(0.2)  # 과도한 요청 방지

df_2["1년뒤_실제종가"] = actual_prices


In [4]:
df_2

,제목,증권사,작성일,목표가,투자의견,작성일_dt,1년뒤_예측일,1년뒤_실제종가
0,2Q24 Pre: 시밀러 본업성장도 견조,DS투자증권,24.07.11,"270,000",매수,2024-07-11,2025-07-11,178600
1,반격의 거인,미래에셋증권,24.07.01,"280,000",매수,2024-07-01,2025-07-01,160400
2,우주의 기운이 셀트리온에게 모인다,DS투자증권,24.06.12,"270,000",매수,2024-06-12,2025-06-12,163900
3,간담회 키워드: 짐펜트라 런칭 성과 그리고 M..,하나증권,24.06.12,"250,000",Buy,2024-06-12,2025-06-12,163900
4,25년부터 이익의 레벨이 달라진다,하나증권,24.05.31,"250,000",Buy,2024-05-31,2025-05-31,155077
5,합병의 효과를 확인할 준비,하이투자증권,24.05.30,"230,000",Buy,2024-05-30,2025-05-30,155077
6,"DDW 2024 후기, 직접 확인하고 왔어요!",키움증권,24.05.27,"220,000",Buy,2024-05-27,2025-05-27,149875
7,"[2024 DDW] A Quick Recap: 짐펜트라, 시간의..",유진투자증권,24.05.24,"250,000",Buy,2024-05-24,2025-05-24,147371
8,시밀러 시장 내 차별화된 경쟁력 보유,유안타증권,24.05.23,"230,000",Buy,2024-05-23,2025-05-23,147082
9,1Q24 & NDR Re. 예상보다 빠른 매출원가율 절..,교보증권,24.05.13,"220,000",Buy,2024-05-13,2025-05-13,148623


# 오차율, 오차율 절댓값 구하기

In [5]:
# 1. 목표가 문자열 → 수치형으로 변환
def parse_price(price_text):
    if not price_text:
        return None
    try:
        return int(price_text.replace(",", "").replace("원", "").strip())
    except:
        return None

df_2["목표가(수치)"] = df_2["목표가"].apply(parse_price)

# 2. 오차율 계산: (목표가 - 실제종가) / 실제종가 * 100
def calculate_error(target, actual):
    try:
        if pd.notnull(target) and pd.notnull(actual) and actual != 0:
            return round((target - actual) / actual * 100, 2)
        else:
            return None
    except:
        return None

df_2["오차율(%)"] = df_2.apply(lambda row: calculate_error(row["목표가(수치)"], row["1년뒤_실제종가"]), axis=1)

df_2

# 절댓값 기준 오차율 낮은 순서로 정렬
df_2["오차율_절댓값"] = df_2["오차율(%)"].abs()
df_2 = df_2.sort_values("오차율_절댓값", ascending=True)

df_2

,제목,증권사,작성일,목표가,투자의견,작성일_dt,1년뒤_예측일,1년뒤_실제종가,목표가(수치),오차율(%),오차율_절댓값
34,"실적 소폭 하회, 모멘텀은 유효",하이투자증권,23.07.26,"200,000",Buy,2023-07-26,2024-07-26,183990,200000.0,8.70,8.70
32,2Q23 Review: 합병을 통한 기업가치 레벨 업 ..,유진투자증권,23.08.16,"200,000",Buy,2023-08-16,2024-08-16,181415,200000.0,10.24,10.24
33,버팀목이 된 신제품생산,키움증권,23.08.16,"200,000",Buy,2023-08-16,2024-08-16,181415,200000.0,10.24,10.24
13,"24년 실적은 상저하고, 하반기 짐펜트라 출시..",대신증권,24.03.06,"200,000",Buy,2024-03-06,2025-03-06,181180,200000.0,10.39,10.39
28,2Q23Re: 모멘텀은 아직 유효하다,DS투자증권,23.08.16,"205,000",Buy,2023-08-16,2024-08-16,181415,205000.0,13.00,13.00
22,신제품 출하로 호실적 기록,키움증권,23.11.08,"190,000",Buy,2023-11-08,2024-11-08,166971,190000.0,13.79,13.79
30,2Q23 Re: 신규 파이프라인으로 수익성 유지,하나증권,23.08.16,"210,000",Buy,2023-08-16,2024-08-16,181415,210000.0,15.76,15.76
31,지금 주가에서는 안 살 이유가 없다,SK증권,23.08.16,"210,000",매수,2023-08-16,2024-08-16,181415,210000.0,15.76,15.76
19,24년 합병 셀트리온 출범,대신증권,23.12.01,"200,000",Buy,2023-12-01,2024-12-01,169823,200000.0,17.77,17.77
25,Issue Comment. 합병 전 or 합병 후 언제 사..,교보증권,23.08.18,"220,000",Buy,2023-08-18,2024-08-18,181783,220000.0,21.02,21.02


# 증권사별로 모으기

In [6]:
# 1. NaN 제거
df_valid = df_2[df_2["오차율(%)"].notnull()]

# 2. 증권사별 통계 집계
broker_error = (
    df_valid.groupby("증권사")["오차율(%)"]
    .agg([
        ("리포트_수", "count"),
        ("평균_오차율(%)", "mean"),
        ("평균_절댓값_오차율(%)", lambda x: x.abs().mean())
    ])
    .reset_index()
)

# 3. 리포트 수 기준 필터링 (선택 사항)
broker_error = broker_error[broker_error["리포트_수"] >= 1]

# 4. 절댓값 오차율 기준 정렬
broker_error_sorted = broker_error.sort_values("평균_절댓값_오차율(%)")

# 5. 출력
display(broker_error_sorted)


,증권사,리포트_수,평균_오차율(%),평균_절댓값_오차율(%)
1,SK증권,3,19.903333,19.903333
3,대신증권,3,28.026667,28.026667
9,하이투자증권,2,28.505000,28.505000
2,교보증권,7,29.452857,29.452857
7,키움증권,4,29.662500,29.662500
8,하나증권,4,38.817500,38.817500
0,DS투자증권,3,42.970000,42.970000
6,유진투자증권,4,48.075000,48.075000
5,유안타증권,1,56.380000,56.380000
4,미래에셋증권,1,74.560000,74.560000


# EWM 오차율 구하기

In [7]:
# 1) NaN 제거 + 날짜형
df_valid = df_2[df_2["오차율(%)"].notnull()].copy()
df_valid["작성일_dt"] = pd.to_datetime(df_valid["작성일_dt"])

# 2) 증권사별 EWM(날짜 가중) 계산
HL_DAYS = 90  # 반감기 90일

def calc_ewm(g):
    g = g.sort_values("작성일_dt").copy()
    g["오차율_EWM"] = g["오차율(%)"].abs().ewm(
        halflife=pd.Timedelta(days=HL_DAYS),
        times=g["작성일_dt"],
        adjust=True   # ← ★ 여기! times를 쓰면 adjust=True 이어야 함
    ).mean()
    return g

df_ewm = df_valid.groupby("증권사", group_keys=False).apply(calc_ewm)

# 3) 증권사별 최종 요약(가장 최신 EWM을 대표값으로)
broker_error = (
    df_ewm.sort_values("작성일_dt")
          .groupby("증권사")
          .agg(
              리포트_수=("오차율(%)", "count"),
              EWM_오차율=("오차율_EWM", "last")
          )
          .sort_values("EWM_오차율")
          .reset_index()
)

display(broker_error)



C:\Users\chloe\AppData\Local\Temp\ipykernel_30592\93774328.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_ewm = df_valid.groupby("증권사", group_keys=False).apply(calc_ewm)


,증권사,리포트_수,EWM_오차율
0,SK증권,3,20.334784
1,대신증권,3,33.460920
2,교보증권,7,35.313228
3,키움증권,4,41.643709
4,하이투자증권,2,44.954017
5,하나증권,4,52.194505
6,DS투자증권,3,55.348731
7,유안타증권,1,56.380000
8,유진투자증권,4,61.523264
9,미래에셋증권,1,74.560000
